# Parte 1. Análisis de sentimiento

## Apartado a)

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split

from sklearn import tree
from sklearn import metrics


file = open("yelp_labelled.txt")

corpus = file.readlines() 

data = []
result = []
for i in range(len(corpus)):
    data.append(corpus[i].split("\t")[0])
    result.append(int(corpus[i].split("\t")[1].split("\n")[0]))
               
train, test, y_train, y_test = train_test_split(data, result, test_size=0.25, random_state=11) 

In [2]:
clasificadores = [] #para guardar los resultados

### Bolsa de palabras binaria con monograma

In [3]:
vectorBM = CountVectorizer(binary = True, ngram_range = (1,1), stop_words = "english")
vectorBMarr = vectorBM.fit_transform(train).toarray()

#para debugear
#print(vectorBM.fit_transform(train).toarray())
#print(vectorBM.get_feature_names())



#### Clasificador NaiveBayes

In [4]:
cnb = BernoulliNB()
cnb.fit(vectorBMarr, y_train)

y_result = cnb.predict(vectorBM.transform(test))

print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("NB binario con monograma:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.736


#### Árbol de decisión

In [5]:
cad = tree.DecisionTreeClassifier()
cad = cad.fit(vectorBMarr,y_train)

y_result = cad.predict(vectorBM.transform(test))
                       
print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("AD binario con monograma:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.708


### Bolsa de palabras binaria con bigrama

In [6]:
vectorBB = CountVectorizer(binary = True, ngram_range = (1,2), stop_words = "english")
vectorBBarr = vectorBB.fit_transform(train).toarray()

#### Clasificador NaiveBayes

In [7]:
cnb = BernoulliNB()
cnb.fit(vectorBBarr, y_train)

y_result = cnb.predict(vectorBB.transform(test))

print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("NB binario con bigrama:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.744


#### Árbol de decisión

In [8]:
cad = tree.DecisionTreeClassifier()
cad = cad.fit(vectorBBarr,y_train)

y_result = cad.predict(vectorBB.transform(test))
                       
print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("AD binario con bigrama:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.716


### Bolsa de palabras TF/IDF con monograma

In [9]:
vectorNBM = CountVectorizer(binary = False, ngram_range = (1,1), stop_words = "english")

pipe = Pipeline([('count', vectorNBM),('tfid', TfidfTransformer())]).fit(train)
pipe['count'].transform(train).toarray()

pipe['tfid'].idf_

vectorNBMarr = pipe.transform(train).toarray()

#### Clasificador NaiveBayes

In [10]:
cnb = BernoulliNB()
cnb.fit(vectorNBMarr, y_train)

y_result = cnb.predict(vectorNBM.transform(test))

print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("NB TFIDF con monograma:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.736


#### Árbol de decisión

In [11]:
cad = tree.DecisionTreeClassifier()
cad = cad.fit(vectorNBMarr,y_train)

y_result = cad.predict(vectorNBM.transform(test))
                       
print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("AD TFIDF con monograma:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.704


### Bolsa de palabras TF/IDF con bigrama

In [12]:
vectorNBB = CountVectorizer(train, binary = False, ngram_range = (1,2), stop_words = "english")

pipe = Pipeline([('count', vectorNBB),('tfid', TfidfTransformer())]).fit(train)
pipe['count'].transform(train).toarray()

pipe['tfid'].idf_

vectorNBBarr = pipe.transform(train).toarray()

#### Clasificador NaiveBayes

In [13]:
cnb = BernoulliNB()
cnb.fit(vectorNBBarr, y_train)

y_result = cnb.predict(vectorNBB.transform(test))

print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("NB TFIDF con bigrama:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.744


#### Árbol de decisión

In [14]:
cad = tree.DecisionTreeClassifier()
cad = cad.fit(vectorNBBarr,y_train)

y_result = cad.predict(vectorNBB.transform(test))
                       
print("Accuracy:",metrics.accuracy_score(y_test, y_result))
clasificadores.append(("AD TFIDF con bigrama:",metrics.accuracy_score(y_test, y_result)))

Accuracy: 0.74


#### Resultados

In [15]:
sorted(clasificadores ,key = lambda x : x[1], reverse = True)

[('NB binario con bigrama:', 0.744),
 ('NB TFIDF con bigrama:', 0.744),
 ('AD TFIDF con bigrama:', 0.74),
 ('NB binario con monograma:', 0.736),
 ('NB TFIDF con monograma:', 0.736),
 ('AD binario con bigrama:', 0.716),
 ('AD binario con monograma:', 0.708),
 ('AD TFIDF con monograma:', 0.704)]

## 1.Analiza la precisión y la exhaustividad de cada clasificador en cada una de las clases (opiniones positivas y negativas). 

In [16]:
#lista.count("item") devuelve nummero de veces aparece item en lista

#Precision: de todos los clasificados, cuantos son reales
#reales ^ identificados por clasificador / identificados por clasificador 



#Exhaustividad: de todos los reales, cuantos ha recuperado
#reales ^ identificados por clasificador / reales

### Para cada clasificador, ¿tiene un comportamiento homogéneo a la hora de clasificar ambas clases? 

### ¿Cuáles son las fortalezas y debilidades de cada uno de los clasificadores? 

### ¿Hay algún clasificador que sea mejor que el otro en todo?  

### ¿Coinciden ambos clasificadores a la hora de clasificar mejor una clase que la otra? 

## Pinta los 8 primeros niveles del árbol de decisión y comenta lo que ves. 

In [17]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
#plt.figure(figsize=(15,15))
#plot_tree(cad, filled=True, feature_names=vectorBM.get_feature_names(), rounded=True, max_depth = 8)
#plt.show()

### ¿Qué estructura tiene el árbol? 

### ¿Cómo interpretas los niveles que has pintado? ¿tienen algún sentido con respecto a la tasa de aciertos, o la precisión y exhaustividad del clasificador? 

### ¿Hay nodos impuros? 

##  Por cada clasificador identifica 2 críticas que hayan sido falsas positivas (malas críticas calificadas como buenas) y 2 críticas que han sido falsas negativas (buenas críticas clasificadas como malas). Analiza tanto su texto original, como el vector de palabras resultante (solamente los términos activos).

### ¿Por qué crees que ha fallado el clasificador en cada uno de los casos? 

### ¿Se te ocurre alguna idea sobre cómo mejorar el clasificador de sentimiento?